### 用爬蟲抓取已發行股數資料

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

### 建立抓取資料函數

> 這邊會需要注意兩個地方

>> 1.用requests搜尋網址，BeautifulSoup找到資料所在的位置。

>> 2.因為是同一個網址內的搜尋欄位(輸入ticker)，所以一定要加timesleep

In [ ]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': 'https://mops.twse.com.tw/'
    }

proxies = {
    'https': 'http://19999990'
}

In [ ]:
def get_data_df(url, stockNo):
    form_data = {
        'encodeURIComponent': 1,
        'step': 1,
        'firstin': 1,
        'off': 1,
        'queryName': 'co_id',
        'inpuType': 'co_id',
        'TYPEK': 'all',
        'isnew': False,
        'co_id': stockNo,
        'date1': 100,
        'date2': 110,
        'qryType': 1
    }

    # 使用 session 保留會話
    session = requests.Session()
    session.headers.update(headers)

    r = requests.post(url, form_data,proxies=proxies)
    r.raise_for_status()  # 確保 HTTP 請求成功
    soup = BeautifulSoup(r.text, 'html.parser')
    tables = soup.find_all('table')
    
    ## 接下來就是從html裡面找資料

    if tables:
        table = tables[0]  # 本網頁的第一個表格
        rows = table.find_all('tr')  # 從所有 row 裡面找資料
    # 繼續處理資料...
    else:
        print(f"未找到任何表格，請檢查網址或請求參數是否正確。")
        return pd.DataFrame()
    # 用於儲存抓到的數據
    data = []
    
    for row in rows:
        th = row.find('th', class_='dColor nowrap')
        if th and ("已發行普通股數或TDR原股發行股數" in th.text):
            td = row.find('td', class_='lColor')
            if td:
                # 將結果存入列表中
                data.append([stockNo, td.text.strip()])
    
    # 將數據轉換為 DataFrame
    if data:
        df = pd.DataFrame(data, columns=['證券代碼', '已發行普通股數或TDR原股發行股數'])
    
    # 如果跑的太頻繁，或是股票代碼有錯的話，就會顯示這一句
    else: 
        print(f"股票代碼 {stockNo} 未找到相關數據。")
        df = pd.DataFrame()
    
    return df

### 用for迴圈抓取所需要的資料

In [ ]:
try:
    # 從 Excel 文件中讀取股票代碼列表
    stock_list_df = pd.read_excel('證券代碼(上市櫃普通股).xlsx')
    stock_list = stock_list_df['公司代號'].tolist()

    # 所有股票的數據並合併為一個 DataFrame
    all_data = pd.DataFrame()

    for stock in stock_list:

        try:
            df = get_data_df("https://mopsov.twse.com.tw/mops/web/t05st03", stock)
            
            if not df.empty:
                print(f"成功抓取股票代碼: {stock}")
                all_data = pd.concat([all_data, df], ignore_index=True)

            time.sleep(random.randint(1, 5))
        except Exception as e:
            print(f"抓取股票代碼 {stock} 時出現錯誤: {e}")
            # 繼續抓取其餘股票的數據

    # 將合併後的 DataFrame 存儲到 Excel 文件中
    if not all_data.empty:
        all_data.to_excel('已發行普通股數或TDR原股發行股數.xlsx', index=False)
        print("資料已存成 Excel 檔案: 已發行普通股數或TDR原股發行股數.xlsx")
    else:
        print("未找到任何有效數據，Excel 檔案未創建。")
        
except Exception as e:
    # 若程式因中斷連線或其他問題停止，保存現有的資料
    if not all_data.empty:
        all_data.to_excel('部分已發行普通股數或TDR原股發行股數.xlsx', index=False)
        print(f"程式執行中止，已保存部分資料: 部分已發行普通股數或TDR原股發行股數.xlsx\n錯誤訊息: {e}")
    else:
        print(f"程式執行中止且未找到任何有效數據。錯誤訊息: {e}")